## Preparing train data
    To check for overfitting

In [1]:
# Loading the test data and making it into a dictionary for faster access
import pandas as pd
df = pd.read_csv('Whole_dataset1.txt')
df = df.sort_values(by='CustomerID')
cust_id = df.CustomerID.tolist()
flag = cust_id[0]; count=1
for i in range(len(cust_id)):
    if(flag==cust_id[i]):
        cust_id[i]=count
    else:
        flag = cust_id[i]
        count+=1
        cust_id[i]=count
df['CustomerID'] = cust_id
df = df.sort_values(by='Date')
l = int(0.7*df.shape[0])
df = df[:l]
mov_id = df.MovieID.tolist()
cust_id = df.CustomerID.tolist()
score = df.Score.tolist()
tr_data = {}
for i in range(len(score)):
    tr_data[(cust_id[i],mov_id[i])] = score[i]
del df,score,cust_id,mov_id

## Preparing the test data

In [2]:
# Loading the test data and making it into a dictionary for faster access
import pandas as pd
df = pd.read_csv('Whole_dataset1.txt')
df = df.sort_values(by='CustomerID')
cust_id = df.CustomerID.tolist()
flag = cust_id[0]; count=1
for i in range(len(cust_id)):
    if(flag==cust_id[i]):
        cust_id[i]=count
    else:
        flag = cust_id[i]
        count+=1
        cust_id[i]=count
df['CustomerID'] = cust_id
df = df.sort_values(by='Date')
l = int(0.7*df.shape[0])
df = df[l:]
mov_id = df.MovieID.tolist()
cust_id = df.CustomerID.tolist()
score = df.Score.tolist()
ts_data = {}
for i in range(len(score)):
    ts_data[(cust_id[i],mov_id[i])] = score[i]
del df,score,cust_id,mov_id

## Loading the matrix

In [3]:
import pickle
with open('sparse_matrix.txt','rb') as fp:
    sp_matrix = pickle.load(fp)
fp.close()
sp_matrix

<470759x4500 sparse matrix of type '<class 'numpy.float64'>'
	with 24053764 stored elements in Compressed Sparse Row format>

## NMF decomposition

In [4]:
from sklearn.decomposition import NMF
model = NMF(n_components=10,init='random')
U = model.fit_transform(sp_matrix)
M = model.components_
U.shape, M.shape

((470759, 10), (10, 4500))

In [5]:
with open('Matrix10_UM.pkl','wb') as fm:
    pickle.dump((U,M),fm)
fm.close()

## RMSE on test data

In [6]:
import math
import numpy as np
e = 0
for i in ts_data.keys():
    row,col = i
    R = ts_data[(row,col)]
    R_h = np.dot(U[row,:],M[:,col])
    #print (R,R_h)
    e = e + math.pow((R-R_h),2)
math.sqrt(e/len(ts_data.keys()))

3.6462575791188945

## RMSE on train data

In [7]:
import math
import numpy as np
e = 0
for i in tr_data.keys():
    row,col = i
    R = tr_data[(row,col)]
    R_h = np.dot(U[row,:],M[:,col])
    #print (R,R_h)
    e = e + math.pow((R-R_h),2)
math.sqrt(e/len(ts_data.keys()))

4.07598747233226

## Observations(on test data):
    n=10, rmse ~ 3.645
    n=20, rmse ~ 3.653
    n=30, rmse ~ 3.66
    n=50, rmse ~ 3.68
    n=2,  rmse ~ 3.66
Because of these following observations, n_components was kept at 10